In [17]:
import sys
import soundcloud

In [18]:
from functools import partial
from utils import get_results, handle_http_errors

In [19]:
def getAttr(resource, attr):
#   if hasattr(resource, 'user'): return resource.user[attr]
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getAttr, attr='username')
getid = partial(getAttr, attr='id')
getUserid = partial(getAttr, attr='user_id')

In [20]:
# A global artist graph used to iterate through the various algorithms.
# Each node is artist id, with edges weighted by activity between then.
#profileGraph = nx.MultiDiGraph()

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')

name = input("Enter a soundcloud artist to analyze: ")


Enter a soundcloud artist to analyze: Sybyr


In [21]:
# Artist of interest
search = client.get('/users/', q = name)[0]

In [23]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'https://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/07/17 04:43:29 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 436,
  'public_favorites_count': 0,
  'followers_count': 51759,
  'followings_count': 22,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 101,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count':

In [24]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr


In [145]:
id2username_cache = {}

In [146]:
@handle_http_errors
def id2username(uid, kind='users'):
    global id2username_dict
    username = id2username_cache.get(uid, None)
    if username is not None: return username

    # username is none, we don't have it in cache
    uid_str = str(uid)
    get = '/{}/{}'.format(kind, uid_str)
    result = client.get(get)
    if kind in ['tracks','comments']:
        username = result.user['username']
    else:
        username = result.username
    # encode it correctly
    #username = str(username.encode('utf-8'))
    #print(username)
    id2username_cache[uid] = username
    return username

# Adding more functions

In [147]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollows(profile):
    # get all profiles a user follows
    follows = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return follows

@handle_http_errors
def getFollowers(profile):
    # get all profiles that follow a user
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getUserFavorites(profile):
    user_favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return user_favorites

@handle_http_errors
def getUserComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

@handle_http_errors
def getTrackComments(track):
    tracks = get_results(client, '/tracks/{0:s}/comments/'.format(str(track)))
    return tracks

In [148]:
# list of profiles to query
profiles_to_query = [search.id]

In [149]:
profiles_to_query

[13377746]

In [150]:
depth = 1
i = 0

# list of profiles we could not query
unavailable_profiles = []

In [151]:
for t in range(depth):

    print("Iteration " + str(t))

    profiles_to_query = list(set(profiles_to_query))

    for profile in profiles_to_query:
        username = id2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))

            encountered_follows = getFollows(profile)
            print("encountered follows: " + ", ".join([getUsername(user) if isinstance(getUsername(user), str) else str(getid(user)) for user in encountered_follows]))
            
            encountered_user_favorites = getUserFavorites(profile)
            print("encountered user favorites: " + ", ".join([getUsername(track) if isinstance(getUsername(track), str) else str(id2username(track.id,'tracks')) for track in encountered_favorites]))
            
            encountered_user_comments = getUserComments(profile)
            print("encountered user comments: " + ", ".join([getUsername(comment) if isinstance(getUsername(comment), str) else str(id2username(comment.track_id,'tracks')) for comment in encountered_user_comments]))#([str(comment.__dict__) for comment in encountered_comments]))#
            
            encountered_followers = getFollowers(profile)
            print("encountered followers: " + ", ".join([getUsername(user) if isinstance(getUsername(user), str) else str(getid(user)) for user in encountered_followers]))

            encountered_tracks = getTracks(profile)
            
            encountered_track_comments = []
            
            for track in encountered_tracks:
                print("getting comments from user track, {}".format(track.title))
                encountered_track_comments += getTrackComments(track.id)
                
            print(encountered_track_comments[0].__dict__)
            
            print("encountered track comments: " + ", ".join([getUsername(comment) if isinstance(getUsername(comment), str) else comment.user['username'] for comment in encountered_track_comments]))#([str(comment.__dict__) for comment in encountered_track_comments]))#

Iteration 0
	 Enqueueing: Sybyr (13377746)
getting comments from user track, ProdbyLandfill.com (feat. Landfill)
getting comments from user track, Do It Again. Prod Kiddbeatz
getting comments from user track, Back Prod Babe
getting comments from user track, Let The Church Say Bang ProdbyLandfill.com
getting comments from user track, Level - Headed Prod Kiddbeatz
getting comments from user track, Unknowns ProdbyLandfill.com
getting comments from user track, Black & Mild Spirits ProdbyLandfill.com
getting comments from user track, Sleepy Cherry Prod Babe
getting comments from user track, Land Of The Land ProdbyLandfill.com
getting comments from user track, Bipolar Unstable Soldier Prod Kiddbeatz
getting comments from user track, Penny Tail 2 ProdbyLandfill.com
getting comments from user track, Not Having It Prod MmmYera
getting comments from user track, Gasping For The Final Time ProdbyLandfill.com
getting comments from user track, IDHTSLTASMD
getting comments from user track, Polyester 

getting comments from user track, Golf Course
getting comments from user track, Born After The Golden Era
getting comments from user track, Cycle
getting comments from user track, A Real Leaking Head
getting comments from user track, 2 For 2
getting comments from user track, Landfill - "reflective polyester" *Lower Yuur Volume*
getting comments from user track, D.E. (Prod. Landfill)
getting comments from user track, [a Thunderstorm Suddenly Appeared] (PROD. @LORDLANDFILL)
getting comments from user track, Full Stream - Sybyrmusic.com
getting comments from user track, WHAT DO YOU SERIOUSLY WANT? (NEW EP ON BANDCAMP JULY 21ST)
getting comments from user track, ProdbyLandfill.com ^leak$ '19 / 9696 - yingyang
getting comments from user track, Gotta Serve Folks (Prod. BruhManeGod)
getting comments from user track, Hate My New Shit (Prod. Shark)
getting comments from user track, I Said I'm Done Playing (Prod. Landfill)
getting comments from user track, Needa Shut The Fuck Up (Prod. Landfill)